@Olga Zagovora, zagovora@uni-koblenz.de, 06.12.16

# Example of progress bars

## Variant 1

In [1]:
#just write your own
import sys
def progress_bar(seq, every=10):
    for i, item in enumerate(seq):
        if i % every ==0:
            print >>sys.stderr, i,
        yield i

In [3]:
#Example:
import time

all_urls=["a"]*100
for url in progress_bar(all_urls):
    print "a"
    time.sleep(0.1)
    

a
a
a
a
a
a
a
a
a
a

 0 


a
a
a
a
a
a
a
a
a
a

10 


a
a
a
a
a
a
a
a
a
a

...

70 


a
a
a
a
a
a
a
a
a
a

80 


a
a
a
a
a
a
a
a
a
a


90

## Variant 2

In [5]:
#use incorporated progressbars
import time
def cool_progress_bar(seq,every=10):
    from ipywidgets import IntProgress
    from IPython.display import display
    progress = IntProgress(min=0,max=len(seq), value=0)
    display(progress)
    
    for i, item in enumerate(seq):
        if i % every ==0:
            progress.value=i
        yield item

In [6]:
all_urls=["a"]*1000
for url in cool_progress_bar(all_urls, every=10):
    #print url
    time.sleep(0.01)

For even more cooler stuff have a look at https://github.com/alexanderkuk/log-progress 

# Multithreading

In [7]:
def coolest_progress_bar(seq,every=10):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display
    progress = IntProgress(min=0,max=len(seq), value=0)
    progress.bar_style = 'info'
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)
    try:
        for i, item in enumerate(seq,1):
            if i== 1 or i % every == 0:
                progress.value = i
                label.value = '{index} / {size}'.format(index=i,size=len(seq))
            yield item
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = i
        label.value = str(i)


In [12]:
from IPython.lib import backgroundjobs as bg

def do_smth(values):
    for url in values:
        time.sleep(0.1)

all_urls=["a"]*100
jobs=bg.BackgroundJobManager()
jobs.new(do_smth,coolest_progress_bar(all_urls, every=1))#name_of_function, parameter_of_function
jobs.new(do_smth,coolest_progress_bar(all_urls, every=1))

Starting job # 0 in a separate thread.
Starting job # 2 in a separate thread.


<BackgroundJob #2: <function do_smth at 0x00000000044AD978>>

In [10]:
jobs.status()

Running jobs:
0 : <function do_smth at 0x00000000044ADE48>
2 : <function do_smth at 0x00000000044ADE48>



# how to kill?

In [11]:
def kill_thread(thread):
    import ctypes
    id = thread.ident
    code = ctypes.pythonapi.PyThreadState_SetAsyncExc(ctypes.c_long(id),ctypes.py_object(SystemError))
    if code == 0:
        raise ValueError('invalid thread id')
    elif code != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(ctypes.c_long(id),ctypes.c_long(0))
        raise SystemError('PyThreadState_SetAsyncExc failed')

In [13]:
kill_thread(jobs.all[2])

In [14]:
for thread in jobs.running:
    kill_thread(thread)

# Chunks

In [15]:
chunks=[all_urls[:33],all_urls[34:66],all_urls[67:]]

jobs=bg.BackgroundJobManager()
for chunk in chunks:
    jobs.new(do_smth,coolest_progress_bar(all_urls, every=1))

Starting job # 0 in a separate thread.
Starting job # 2 in a separate thread.
Starting job # 3 in a separate thread.


In [12]:
def get_chunks(seq,number_of_chunks):
    from itertools import repeat
    
    number_of_chunks=min(number_of_chunks,len(seq))
    chunks=[[] for i in repeat(None, number_of_chunks)]
    for i, item in enumerate(seq):
        chunks[i % number_of_chunks].append(item) 
    return chunks

for chunk in get_chunks(all_urls,4):
    jobs.new(do_smth,coolest_progress_bar(all_urls, every=1))

Starting job # 4 in a separate thread.
Starting job # 5 in a separate thread.
Starting job # 6 in a separate thread.
Starting job # 7 in a separate thread.


In [1]:
from itertools import repeat
import numpy 


n=4
%timeit d = [[] for i in repeat(None, n)]
%timeit d = [[] for x in range(n)]
%timeit d = numpy.empty((n, 0)).tolist()

100000 loops, best of 3: 3.17 µs per loop
100000 loops, best of 3: 2.73 µs per loop
The slowest run took 8.95 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 5.74 µs per loop


# But the global interpreter lock, or GIL!!!

# Another example of multithreading

In [2]:
from IPython.display import display
from ipywidgets import IntProgress

import threading

class App(object):
    def __init__(self, nloops=2000):
        self.nloops = nloops
        self.pb = IntProgress(description='Thread loops', min=0, max=self.nloops)

    def start(self):
        display(self.pb)
        while self.pb.value < self.nloops:
            self.pb.value += 1 
        self.pb.color = 'red'

app = App(nloops=20000)

t = threading.Thread(target=app.start)

t.start()
#t.join()